**Please read the report to find out the working of this codes**

# Loading Libraries

In [2]:
#All the libraries used in this code
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Classifiers

In [3]:
class NB():
  def __init__(self,X,Y):
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3)
    print(X_train.shape,Y_train.shape)
    N,D=X_train.shape
    X_train_1=[]
    X_train_0=[]
    for i in range(N):
      if Y_train[i]==1:
        X_train_1.append(X_train[i])
      else:
        X_train_0.append(X_train[i])
    ones=np.ones((1,D)) #Smoothening
    X_train_1.append(ones)
    X_train_0.append(ones)

    X_train_1=np.array(X_train_1)
    X_train_0=np.array(X_train_0)
    p=X_train_1.shape[0]/(N+2)
    p_1=np.mean(X_train_1,axis=0).reshape(-1)
    p_0=np.mean(X_train_0,axis=0).reshape(-1)
    Y_test_pred=[]
    test_acc=0
    for i in range(X_test.shape[0]):
      p_0_x=1-p
      p_1_x=p
      for j in range(D):
        p_1_x *= (X_test[i][j]*p_1[j] + (1-X_test[i][j])*(1-p_1[j])) 
        p_0_x *= (X_test[i][j]*p_0[j] + (1-X_test[i][j])*(1-p_0[j])) 
      if p_0_x > p_1_x:
        Y_test_pred.append(0)
        if Y_test[i]==0:
          test_acc+=1
      else:
        Y_test_pred.append(1)
        if Y_test[i]==1:
          test_acc+=1
    test_acc/=X_test.shape[0]
    self.Y_train_pred=np.array(Y_test_pred)
    self.test_acc=test_acc
    self.p_1=p_1
    self.p_0=p_0
    self.p=p
    self.N,self.D=N,D
  def predict(self,X_test):
    Y_test_pred=[]
    for i in range(X_test.shape[0]):
      p_0_x=1-self.p
      p_1_x=self.p
      for j in range(self.D):
        p_1_x *= (X_test[i][j]*self.p_1[j] + (1-X_test[i][j])*(1-self.p_1[j])) 
        p_0_x *= (X_test[i][j]*self.p_0[j] + (1-X_test[i][j])*(1-self.p_0[j])) 
      if p_0_x > p_1_x:
        Y_test_pred.append(0)
      else:
        Y_test_pred.append(1)
    Y_test_pred=np.array(Y_test_pred)
    return Y_test_pred

In [4]:
class GNB():
  def __init__(self,X,Y,var):
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
    N,D=X_train.shape
    X_train_1=[]
    X_train_0=[]
    for i in range(N):
      if Y_train[i]==1:
        X_train_1.append(X_train[i])
      else:
        X_train_0.append(X_train[i])
    X_train_1=np.array(X_train_1,dtype=float)
    X_train_0=np.array(X_train_0,dtype=float)
    p=X_train_1.shape[0]/(N)
    mu_1=np.mean(X_train_1,axis=0).reshape(-1)
    mu_0=np.mean(X_train_0,axis=0).reshape(-1)
    if var=='unit':
      cov=np.eye(D,D)
      cov_1=cov
      cov_0=cov
    elif var=='same':
      cov = (X_train_1.shape[0]*np.cov(X_train_1,rowvar=False,bias=True) + X_train_0.shape[0]*np.cov(X_train_0,rowvar=False,bias=True)) / N
      cov_1=cov
      cov_0=cov
    else:
      cov_1=np.cov(X_train_1,rowvar=False,bias=True)
      cov_0=np.cov(X_train_0,rowvar=False,bias=True)
    Y_test_pred=[]
    test_acc=0
    for i in range(X_test.shape[0]):
      p_0_x = (1-p)*np.dot(np.dot(np.transpose(X_test[i]-mu_0),np.linalg.pinv(cov_0)),X_test[i]-mu_0)
      p_1_x = p*np.dot(np.dot(np.transpose(X_test[i]-mu_1),np.linalg.pinv(cov_1)),X_test[i]-mu_1)
      if p_0_x > p_1_x:
        Y_test_pred.append(0)
        if Y_test[i]==0:
          test_acc+=1
      else:
        Y_test_pred.append(1)
        if Y_test[i]==1:
          test_acc+=1
    test_acc/=X_test.shape[0]
    self.Y_test_pred=np.array(Y_test_pred)
    self.test_acc=test_acc
    self.mu_0=mu_0
    self.mu_1=mu_1
    self.cov_0=cov_0
    self.cov_1=cov_1
    self.p=p
    self.N,self.D=N,D
  def predict(self,X_test):
    Y_test_pred=[]
    for i in range(X_test.shape[0]):
      p_0_x = (1-self.p)*np.dot(np.dot(np.transpose(X_test[i]-self.mu_0),np.linalg.pinv(self.cov_0)),X_test[i]-self.mu_0)
      p_1_x = self.p*np.dot(np.dot(np.transpose(X_test[i]-self.mu_1),np.linalg.pinv(self.cov_1)),X_test[i]-self.mu_1)
      if p_0_x > p_1_x:
        Y_test_pred.append(0)
      else:
        Y_test_pred.append(1)
    Y_test_pred=np.array(Y_test_pred)
    return Y_test_pred

In [5]:
class LogisticRegression():
    def __init__(self,X,Y,iters=100,lr=0.01) -> None:
        X=np.concatenate([X,np.ones((X.shape[0],1))],axis=1) #to account for bias
        X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
        Y_train=Y_train.reshape(-1,1)
        W=np.random.rand(X.shape[1],1)
        cost=[]
        for iter in range(iters):
            Y_hat=self.sigmoid(X_train.dot(W))
            cost.append((self.cost(Y_hat,Y_train)))
            W = W - lr*self.grad(X_train,Y_hat,Y_train)
        y_test_pred=self.sigmoid(X_test.dot(W))
        y_test_pred[y_test_pred>0.5]=1.
        y_test_pred[y_test_pred<=0.5]=0.
        y_test_pred=y_test_pred.reshape(-1)
        test_acc=0
        for i,y in enumerate(y_test_pred):
            if y==Y_test[i]:
                test_acc+=1
        test_acc/=Y_test.shape[0]
        self.W=W
        self.test_acc=test_acc
        self.costs=cost
    def cost(self,y_hat,y):
        return -np.average(y*np.log(y_hat+1e-8) + (1-y)*np.log(1-y_hat+1e-8))
    def grad(self,x,y_hat,y):
        return np.dot(x.T,y_hat-y)
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def predict(self,X_test):
        X_test=np.concatenate([X_test,np.ones((X_test.shape[0],1))],axis=1) #to account for bias
        y_test_pred=self.sigmoid(X_test.dot(self.W))
        y_test_pred[y_test_pred>0.5]=1.
        y_test_pred[y_test_pred<=0.5]=0.
        return y_test_pred.reshape(-1)

In [6]:
class KNN():
    def __init__(self,X,Y,k=5) -> None:
        X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
        self.X_train=X_train
        self.X_test=X_test
        self.Y_train=Y_train
        self.Y_test=Y_test
        self.k=k
        y_test_pred=self.predict(X_test)
        test_acc=0
        for i,y in enumerate(y_test_pred):
            if y==Y_test[i]:
                test_acc+=1
        test_acc/=Y_test.shape[0]
        self.test_acc=test_acc
    def predict(self,X_test):
        Y_pred_list=[]
        for i in X_test:
            dists=np.linalg.norm(self.X_train-i,axis=1)
            a=np.argsort(dists)
            ones,zeros=0,0
            for k in range(self.k):
                if self.Y_train[a[k]]==1:
                    ones+=1
                else:
                    zeros+=1
            Y_pred_list.append(1 if ones>zeros else 0)
        return Y_pred_list

The bottom section is only to do preprocessing dont run the code blocs in this section as it will take quite a while to do so. I have created another CSV using the preprocessed CSV that is what we will be using for training and validation

# Making processed CSV

In [ ]:
missing_values = ["n/a", "na", "--" , "empty" , "?" , "??" , "---" , "-", "__" , "___" , "___ ___"," "]
df=pd.read_csv("spam/completeSpamAssassin.csv",na_values=missing_values)
df.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [ ]:
df.isnull().sum()

Unnamed: 0      0
Body          534
Label           0
dtype: int64

In [ ]:
df=df.drop_duplicates()
df=df.dropna()

In [ ]:
df.shape

(5512, 3)

In [ ]:
def preprocess(text):
  text=re.sub("[^a-zA-Z0-9]"," ",text) #Removing unwanted characters from text
  text=text.lower().split() #changing all text to lowercases and splitting them 
  lem=WordNetLemmatizer()
  words=[lem.lemmatize(word) for word in text if word not in stopwords.words('english')]
  return " ".join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


TLDR: Lemmatization is better.

**Note on  Lemmatizer and stemmer:**

Stemming is the process of producing morphological variants of a root/base word.

Stemming is a proccess of chopping of the ends of the word until a stem is reached. This does not pay any attention of take any information from the structure of the language. There are several stemmer algoritms that were developed including porter stemmer and snowball stemmer.

Stemming does not take into consideration of how the word was used. As in does not see if the word was a noun or verb or anything just reduces it to its stem.

That is why lemmetizers are better. Lemmatization applies a morphological analysis to words by taking into account a language's entire lexicon in addition to word reduction. Most people believe lemmatization to be far more informative than simple stemming. Lemmatization does not classify sentences; instead, it analyses the context of a word to determine its part of speech.


[Reference](https://towardsdatascience.com/stemming-vs-lemmatization-2daddabcb221)


In [ ]:
body=df['Body'].to_list()

In [ ]:
text=[]
for i in body:
    text.append(preprocess(i))

In [ ]:
text

['save 70 life insurance spend life quote saving ensuring family financial security important life quote saving make buying life insurance simple affordable provide free access best company lowest rate life quote saving fast easy save money let u help get started best value country new coverage save hundred even thousand dollar requesting free quote lifequote saving service take le 5 minute complete shop compare save 70 type life insurance click free quote protecting family best investment ever make receipt email error wish removed list please click type remove reside state prohibits e mail solicitation insurance please disregard email',
 '1 fight risk cancer http www adclick w p cfm 315 pk0072 slim guaranteed lose 10 12 lb 30 day http www adclick w p cfm 249 pk0073 get child support deserve free legal advice http www adclick w p cfm 245 pk0024 join web fastest growing single community http www adclick w p cfm 259 pk0075 start private photo album online http www adclick w p cfm 283 pk0

In [ ]:
df['preprocessed_text']=text

In [ ]:
df.head()

,Unnamed: 0,Body,Label,preprocessed_text
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1,save 70 life insurance spend life quote saving...
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1,1 fight risk cancer http www adclick w p cfm 3...
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1,1 fight risk cancer http www adclick w p cfm 3...
3,3,##############################################...,1,adult club offer free membership instant acces...
4,4,I thought you might like these:\n1) Slim Down ...,1,thought might like 1 slim guaranteed lose 10 1...


In [ ]:
df.to_csv("spam/processed_data.csv")

# Using processed CSV

Preprocessing takes a lot of time and hence i have cretred another csv called preprocessed_data which we will be using as out dataset.

In [7]:
df=pd.read_csv("spam/processed_data.csv") 

for feature extraction we will be looking at 2 different forms of feature extraction from sklearn package

They will be CountVectorizer and TfidfVectorizer

We will discuss their working briefly

**CountVectorizer:**
This will convert text to vectors. So what will this vector have? it consists of 0s if word is not there and the counts number of times the word appears in the sentence

**TfidfVectorizer:**
This consists of two things one term frequency and the other inverse document frequency. Term frequency stands for the ration of number of times the word has occured in a document(one email in this case) to the total number of words in that document.Inverse document frequency refers to the log of the total number of documents divided by the number of documents that contain the word. The product of these two is assigned to each word. 

Which is better?

From my search on the internet I found that there is no definitive answer to this. But TFIDF tends to serve more information about the word and a little bit of semantic meaning along with it than CountVectorizer. 
.

In [8]:
cv=CountVectorizer(binary=True) #limiting max features for computational reasons
tfidf=TfidfVectorizer(max_features=2000)
Y=df['Label'].to_numpy()
cv=cv.fit(df['preprocessed_text'].to_list())
tfidf=tfidf.fit(df['preprocessed_text'].to_list())
X_cv=cv.transform(df['preprocessed_text'].to_list()).toarray()
X_tfidf=tfidf.transform(df['preprocessed_text'].to_list()).toarray ( )

In [9]:
X_cv.shape

(5509, 62352)

There are a total of 62352 unique words in thw corpus

In [10]:
nb=NB(X_cv,Y)

(3856, 62352) (3856,)


C:\Users\user\AppData\Local\Temp\ipykernel_5068\3274300531.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_1=np.array(X_train_1)
C:\Users\user\AppData\Local\Temp\ipykernel_5068\3274300531.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_0=np.array(X_train_0)


KeyboardInterrupt: 

In [ ]:
nb.test_acc

0.9280096793708409

In [ ]:
k=KNN(X_tfidf,Y)

In [11]:
k.test_acc

0.632486388384755

In [12]:
l=LogisticRegression(X_cv,Y)

C:\Users\user\AppData\Local\Temp\ipykernel_12416\3312228540.py:29: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


In [13]:
l.test_acc

0.985480943738657

In [14]:
l=LogisticRegression(X_tfidf,Y)

In [15]:
l.test_acc

0.9736842105263158

In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(X_cv,Y,test_size=0.2)

In [1]:
from sklearn import svm
svm=svm.SVC(kernel='linear')
svm.fit(X_train,Y_train)
Y_pred=svm.predict(X_test)
test_acc=0
for i,y in enumerate(Y_pred):
    if y==Y_test[i]:
        test_acc+=1
test_acc/=Y_test.shape[0]
test_acc

NameError: name 'X_train' is not defined

In the above cell block i have tried out SVM for classification and it was performing decently well. But logistic regression was outperforming it by a little margin. 

Looking at the above results its pretty clear that Logistic Regression is working best. So will write a funtion below which will use logistic regression to classify the given test data. 

In [12]:
l=list(range(1000,10000,1000))
for i in l:
    cv=CountVectorizer(max_features=i,binary=True) #limiting max features for computational reasons
    cv_t=CountVectorizer(max_features=i)
    tfidf=TfidfVectorizer(max_features=i)
    cv=cv.fit(df['preprocessed_text'].to_list())
    cv_t=cv_t.fit(df['preprocessed_text'].to_list())
    tfidf=tfidf.fit(df['preprocessed_text'].to_list())
    X_cv=cv.transform(df['preprocessed_text'].to_list()).toarray()
    X_cv_t=cv_t.transform(df['preprocessed_text'].to_list()).toarray()
    X_tfidf=tfidf.transform(df['preprocessed_text'].to_list()).toarray()
    cv_acc=cvt_acc=tfidf_acc=0
    for _ in range(5):
        l_cv=LogisticRegression(X_cv,Y)
        l_cv_t=LogisticRegression(X_cv_t,Y)
        l_tifdf=LogisticRegression(X_tfidf,Y)
        cv_acc+=l_cv.test_acc
        cvt_acc+=l_cv_t.test_acc
        tfidf_acc+=l_tifdf.test_acc
    cv_acc/=5
    cvt_acc/=5
    tfidf_acc/=5
    print(f"for CV:{cv_acc} for CV_T:{cvt_acc} for TFIDF:{tfidf_acc} fir i:{i}")

C:\Users\user\AppData\Local\Temp\ipykernel_6500\3312228540.py:29: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


for CV:0.9800362976406534 for CV_T:0.9718693284936479 for TFIDF:0.9771324863883848 fir i:1000
for CV:0.9831215970961887 for CV_T:0.9767695099818512 for TFIDF:0.9718693284936479 fir i:2000
for CV:0.980399274047187 for CV_T:0.9769509981851181 for TFIDF:0.9736842105263157 fir i:3000
for CV:0.9845735027223231 for CV_T:0.9798548094373866 for TFIDF:0.9729582577132486 fir i:4000
for CV:0.9825771324863883 for CV_T:0.9749546279491833 for TFIDF:0.9733212341197822 fir i:5000
for CV:0.983847549909256 for CV_T:0.9778584392014519 for TFIDF:0.9724137931034482 fir i:6000
for CV:0.9854809437386571 for CV_T:0.9776769509981851 for TFIDF:0.9671506352087114 fir i:7000
for CV:0.9822141560798547 for CV_T:0.9773139745916515 for TFIDF:0.9695099818511796 fir i:8000
for CV:0.9829401088929218 for CV_T:0.9756805807622504 for TFIDF:0.9702359346642467 fir i:9000


In [11]:
#training with all features
cv=CountVectorizer(max_features=i,binary=True) #limiting max features for computational reasons
cv_t=CountVectorizer()
tfidf=TfidfVectorizer()
cv=cv.fit(df['preprocessed_text'].to_list())
cv_t=cv_t.fit(df['preprocessed_text'].to_list())
tfidf=tfidf.fit(df['preprocessed_text'].to_list())
X_cv=cv.transform(df['preprocessed_text'].to_list()).toarray()
X_cv_t=cv_t.transform(df['preprocessed_text'].to_list()).toarray()
X_tfidf=tfidf.transform(df['preprocessed_text'].to_list()).toarray()
cv_acc=cvt_acc=tfidf_acc=0
for _ in range(5):
    l_cv=LogisticRegression(X_cv,Y)
    l_cv_t=LogisticRegression(X_cv_t,Y)
    l_tifdf=LogisticRegression(X_tfidf,Y)
    cv_acc+=l_cv.test_acc
    cvt_acc+=l_cv_t.test_acc
    tfidf_acc+=l_tifdf.test_acc
cv_acc/=5
cvt_acc/=5
tfidf_acc/=5
print(f"for CV:{cv_acc} for CV_T:{cvt_acc} for TFIDF:{tfidf_acc}")

C:\Users\user\AppData\Local\Temp\ipykernel_6500\3312228540.py:29: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


for CV:0.9800362976406533 for CV_T:0.9784029038112523 for TFIDF:0.9588021778584391


From the above outputwe can see that increasing the number of featues does increase the validation accuracy. But only to a certain extent. We have tried to find the best hyperparameter for training for logistic regression only as it was performing better than rest of the models. From that it can be seen that CountVectorizer with binary=True and max_features=4000

# Trying it out on test data

So the flow would be like this:
1. preprocess the data - cleaning
2. convert into encodings
3. pass it as test dataset to the logistic regression model

**One cell to do them all**

In [1]:
#All the libraries used in this code
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

class LogisticRegression():
    def __init__(self,X,Y,iters=100,lr=0.01) -> None:
        X=np.concatenate([X,np.ones((X.shape[0],1))],axis=1) #to account for bias
        X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
        Y_train=Y_train.reshape(-1,1)
        W=np.random.rand(X.shape[1],1)
        cost=[]
        for iter in range(iters):
            Y_hat=self.sigmoid(X_train.dot(W))
            cost.append((self.cost(Y_hat,Y_train)))
            W = W - lr*self.grad(X_train,Y_hat,Y_train)
        y_test_pred=self.sigmoid(X_test.dot(W))
        y_test_pred[y_test_pred>0.5]=1.
        y_test_pred[y_test_pred<=0.5]=0.
        y_test_pred=y_test_pred.reshape(-1)
        test_acc=0
        for i,y in enumerate(y_test_pred):
            if y==Y_test[i]:
                test_acc+=1
        test_acc/=Y_test.shape[0]
        self.W=W
        self.test_acc=test_acc
        self.costs=cost
    def cost(self,y_hat,y):
        return -np.average(y*np.log(y_hat+1e-8) + (1-y)*np.log(1-y_hat+1e-8))
    def grad(self,x,y_hat,y):
        return np.dot(x.T,y_hat-y)
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def predict(self,X_test):
        X_test=np.concatenate([X_test,np.ones((X_test.shape[0],1))],axis=1) #to account for bias
        y_test_pred=self.sigmoid(X_test.dot(self.W))
        y_test_pred[y_test_pred>0.5]=1.
        y_test_pred[y_test_pred<=0.5]=0.
        return y_test_pred.reshape(-1)

def preprocess(text):
  text=re.sub("[^a-zA-Z0-9]"," ",text) #Removing unwanted characters from text
  text=text.lower().split() #changing all text to lowercases and splitting them 
  lem=WordNetLemmatizer()
  words=[lem.lemmatize(word) for word in text if word not in stopwords.words('english')]
  return " ".join(words)
def run_this(path):
    if path[-1]!='/':
        path+='/'
    emails=os.listdir(path)
    os.path.join(path + emails[0])

    #preprocessing of test data
    test_data=[]
    for email in emails:
        f=open(os.path.join(path,email))
        lines=""
        for line in f:
            lines+=line
        lines=preprocess(lines)
        test_data.append(lines)

    #training model on train data
    #make sure the train data is also in this directory else will throw errors
    df=pd.read_csv("spam\processed_data.csv")
    cv=CountVectorizer(max_features=4000,binary=True) #limiting max features for computational reasons
    Y=df['Label'].to_numpy()
    cv=cv.fit(df['preprocessed_text'].to_list())
    X_cv=cv.transform(df['preprocessed_text'].to_list()).toarray()
    model=LogisticRegression(X_cv,Y,iters=100,lr=0.01)
    print(f"Validataion accuracy of model: {model.test_acc}")
    #Using test data
    X_cv_test=cv.transform(test_data).toarray()
    y_test_preds=model.predict(X_cv_test)
    predictions={}
    for pred,email in zip(y_test_preds,emails):
        predictions[email]=pred
    return predictions

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
X=run_this("TestDataset")

C:\Users\user\AppData\Local\Temp\ipykernel_16188\3823331941.py:47: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-z))


Validataion accuracy of model: 0.984573502722323


In [3]:
X

{'email0.txt': 1.0, 'email01.txt': 0.0, 'email02.txt': 0.0}

When you run 'run_this' you will get a dictionary as output with file names as key and predictions as their values. We have shown an example above